In [8]:
from sys import getsizeof
from random import randint
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

from hdwallet import HDWallet, BIP141HDWallet
from hdwallet.symbols import FOXD
from hdwallet.cryptocurrencies import get_cryptocurrency

import pickledb
from mnemonic import Mnemonic

from src.RPC import Call
from src.database import credentials, mnemonic, settings, sighash

In [11]:
account = 'wallet.0'
sigtypes = sighash
language = settings['language']
password = settings['password']
phrase = mnemonic[account][-1] # type: ignore

In [ ]:
class WalletDB:
    def __init__(self, symbol: str, account: str) -> None:
        self.chindex = 0
        self.symbol = symbol
        self.account = account
        self.accindex = account.split('.')[1]
        self.db = pickledb.load(f'{symbol.lower()}.wallet.json', True)
        self.lenaddrs = self.db.llen(account)
        self.listaddrs = list(self.db.lgetall(account))
        self.lenkeys = self.db.totalkeys()
        self.keys = list(self.db.getall())
        
    def generate_mnemonics(self, amount, offset=0):
        mndb = pickledb.load('src/database/mnemonic.json', True)
        for i in range(amount):
            name = f'wallet.{i + offset}'
            if not mndb.exists(name): mndb.lcreate(name)
            new_mnem = Mnemonic(language).generate()
            mndb.append(name, [new_mnem])

    def get_wallet(self, accindex, addrindex=0) -> HDWallet:
        wallet = BIP141HDWallet(self.symbol, get_cryptocurrency(self.symbol), f"m/44'/175'/{accindex}'/{self.chindex}/{addrindex}")
        return wallet.from_mnemonic(phrase, language, password)

    def get_wallets(self, accindex, amount, offset=0) -> list[HDWallet]: return [
        self.get_wallet(accindex, i + offset) for i in range(amount)]

    def generate_addresses(self, amount=1):
        wallets = self.get_wallets(int(self.account.split('.')[1]), amount, self.lenaddrs)
        addresses = [wallet.p2pkh_address() for wallet in wallets]
        if not self.db.exists(self.account): self.db.lcreate(self.account)
        if self.db.append(self.account, addresses): return addresses

In [ ]:
account, amount = f'account.{0}', 10000
walletdb = WalletDB(FOXD, account)

In [ ]:
# if walletdb.lenaddr < amount:
#     amount = amount
# walletdb.generate_addresses(amount)

def switch_account_wallet(): pass

walletdb.keys

walletdb.lenkeys
getaddress = lambda addr: walletdb.db.lget(account, addr)
# addresses = [getaddress(i+1) for i in range(31)]
# from concurrent.futures import ThreadPoolExecutor, as_completed
# with ThreadPoolExecutor(16) as executor:
#     for i in range(10): # walletdb.lenaddr//50
#         futures = [executor.submit(rpc.getaddressbalance, getaddress(i))]
# 
#         for future in as_completed(futures):
#             print(future.result())

In [ ]:
rpc = Call(credentials['username'], credentials['password'], 8770)

def vins(txids, vouts): return [{'txid': t, 'vout': v} for t, v in zip(txids, vouts)]
def vouts(): return {}
def createtx(vins: list[dict], vouts: dict): return rpc.createrawtransaction(vins, vouts)
def calcfee(txhash: str, feerate=.01): return feerate * getsizeof(txhash) / 1000

In [ ]:
# rpc = RPC(credentials['username'], credentials['password'], 8770)

wallet = walletdb.get_wallet(0, 0)
address = wallet.p2pkh_address()
priv_key = wallet.wif() # rpc.importprivkey(priv_key, '', False)
wallet = walletdb.get_wallet(0, 1)
chaddr = wallet.p2pkh_address()

def send_coin(p2pkh, wif):
    utxo = rpc.getaddressutxos(p2pkh)[0]
    txid = utxo['txid']
    vout = utxo['outputIndex']
    script = utxo['script']
    balance = round(utxo['satoshis'] / 1e8, 8)

    # creates vin and update for signatue
    # vin = (lambda t, v: {'txid': t, 'vout': v})(txid, vout)
    # vin.update(scriptPubKey=script)
    # print(vin)
    
    def calcfee():
        txhash = createtx([{'txid': txid, 'vout': vout}], {})
        sigtx = rpc.signrawtransaction(txhash, [{'txid': txid, 'vout': vout, 'scriptPubKey': script}], [wif], "ALL|ANYONECANPAY")
        bytesize = getsizeof(sigtx['hex'])
        round(.01 * bytesize / 2 / 1000, 8)
        return .01 * rpc.decoderawtransaction(sigtx['hex'])['size'] / 1000
    txfee = calcfee()
    txhash = rpc.createrawtransaction([{'txid': txid, 'vout': vout}], {'FEXoWJvWszsofzwEv4rvKAj54u94jxk9KW': (balance//2)-txfee, chaddr:(balance//2)-txfee})
    sigtx = rpc.signrawtransaction(txhash, [{'txid': txid, 'vout': vout, 'scriptPubKey': script}], [wif], "ALL|ANYONECANPAY")
    if bool(sigtx['complete']): return rpc.sendrawtransaction(str(sigtx['hex']))
# print(rpc.decoderawtransaction(send_coin(address, priv_key)))
# print(send_coin(address, priv_key)) # 4bef8d65e7c223a3f015789cfb3bc250995893c9c9fd198e80e6891d006811e8 # FEXoWJvWszsofzwEv4rvKAj54u94jxk9KW

In [ ]:
# rpc = Call(credentials['username'], credentials['password'], 8770)

# rpc.getaddressbalance(address)
# rpc.decoderawtransaction()
print(rpc.gettransaction('4bef8d65e7c223a3f015789cfb3bc250995893c9c9fd198e80e6891d006811e8'))
_vin = lambda t, v: {'txid': t, 'vout': v}
_vin = _vin(345467, 0)
_vin.update(scriptPubKey=54353)
print(_vin)


In [ ]:
# rpc = Call(credentials['username'], credentials['password'], 8770)



In [ ]:
# rpc = Call(credentials['username'], credentials['password'], 8770)

# addresses = list(db.lgetall(account))
offset = 9000
addresses = [getaddress(i+offset) for i in range(1000)]
bal = rpc.getbalance('')
print(f'{bal}\n{addresses}')

In [ ]:
# rpc = Call(credentials['username'], credentials['password'], 8770)

name = 'wallet.0'
tdb = pickledb.load('tx.json', True)

if not tdb.exists(name): tdb.lcreate(name)



recipients = {address: round(bal/randint(35000, 45000), 8) for address in addresses}

tx = rpc.sendmany('', recipients, 1, '', addresses)
if not tx: pass
else: tdb.append(name, [tx])
print(f'{tx}\n{recipients}')

In [ ]:
# rpc = Call(credentials['username'], credentials['password'], 8770)

rpc.walletlock()

In [ ]:
# rpc = Call(credentials['username'], credentials['password'], 8770)

# txs = tdb.lgetall('wallet.0')
# rpc.gettransaction(txs[-1])['details']

rpc.gettransaction('4bef8d65e7c223a3f015789cfb3bc250995893c9c9fd198e80e6891d006811e8')

In [ ]:
# rpc = Call(Client(), credentials['username'], credentials['password'], 8770)

# addresses = db.get(account)
# 
# 
# 
# 
# denominator = len(addresses) - 1
# recipients = {address: round(balance/denominator, 8) for address in addresses}
# tx = rpc.sendmany(account, recipients, 1, '', addresses)


with ThreadPoolExecutor() as executor: pass

In [ ]:

# def load_db(filename): return pickledb.load(f'{filename}.json', True)
# db = load_db('addresses')


with ThreadPoolExecutor() as executor: pass
# def lexists_seq(name, data):
#     if type(data) is list | tuple:
#         for d in data: 
#             pass
#     
# def lexists_ladd(self, name, data): 
#     if not db.lexists(name, data): db.ladd(name, data)
# def save_addresses(account): rpc.getaddressesbyaccount(account)
# 
# 
# def add_list(self, name, data):
#     if not db.exists(name): db.lcreate(name)
# db.append('joshie', ['2'])
cache = []

# for address in db.lgetall('joshie'):
#     print(address)
# list_accounts(rpc)

In [ ]:
rpc = Call(credentials['username'], credentials['password'], 8770)

file = Path.absolute(Path('foxd_assets.json'))
db = pickledb.load(file, True)
search = 'ENO'

def is_valid_asset(asset):
    if db.exists(asset):
        print(db.get(search))
        return True
    for key, value in rpc.listassets(search, True):
        db.set(key, value)
        if key is search: return True
    return False
is_valid_asset(search)

In [ ]:
from hashlib import new
def ripemd160(data): return new('ripemd160', data)
ripemd160(b'hello').hexdigest()